In [77]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

In [78]:
df = pd.read_csv('/content/creditcard.csv')

In [79]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


**PART1 - PreProcessing**

In [80]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [81]:
df.isnull().sum()

,0
Time,0
V1,0
V2,0
V3,0
V4,0
V5,0
V6,0
V7,0
V8,0
V9,0


In [82]:
df.dropna(inplace=True)

In [83]:
x = df.drop(columns='Class', axis=1)
y = df['Class']

# Ensure all columns in x are numeric, coercing errors to NaN
x = x.apply(pd.to_numeric, errors='coerce')
x.dropna(inplace=True)
y = y[x.index] # Align y with the potentially reduced x after dropping NaNs

In [84]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [85]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

Custom DataSet

In [86]:
class CustomDataSet(Dataset):
  def __init__(self, features, labels):
    self.features = features
    self.labels = labels

  def __len__(self):
    return len(self.features)

  def __getitem__(self, index):
    return self.features[index], self.labels[index]

In [87]:
train_dataset = CustomDataSet(x_train, y_train.values)
test_dataset  = CustomDataSet(x_test, y_test.values)

In [88]:
class MyNN(nn.Module):
  def __init__(self, input_dim, output_dim, num_hidden_layer, neuron_per_layer, Dropout):

    super().__init__()
    layer = []

    for i in range(num_hidden_layer):
      layer.append(nn.Linear(input_dim, neuron_per_layer))
      layer.append(nn.BatchNorm1d(neuron_per_layer))
      layer.append(nn.ReLU())
      layer.append(nn.Dropout(Dropout))
      input_dim = neuron_per_layer

    # The last layer maps from the last hidden layer size to the output_dim
    layer.append(nn.Linear(neuron_per_layer, output_dim))

    self.model = nn.Sequential(*layer)
  def forward(self, x):
    return self.model(x)

In [92]:
def objective(trails):
  num_hidden_layers = trails.suggest_int("hidden_layer", 1, 5)
  neuron_per_layer  = trails.suggest_int("neurons",8, 128, step=8)
  epochs            = trails.suggest_int("epchs", 1, 10)
  learning_rate      = trails.suggest_float("lr", 1e-5, 1e-4, log=True)
  drop_out          = trails.suggest_float("dropout", 0.1, 0.5, step=0.1)
  batch_size        = trails.suggest_categorical("batch_size", [16, 32, 62, 128])
  optimizer_name    = trails.suggest_categorical("optimizer", ['Adam', 'RMSprop', 'SGD'])


  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory = True)
  test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, pin_memory = True)

  input_dim = x_train.shape[1]
  output_dim = 1

  # Pass drop_out rate to MyNN, not batch_size
  model = MyNN(input_dim, output_dim, num_hidden_layers, neuron_per_layer, drop_out).to(device)

  criterion = nn.BCEWithLogitsLoss()

  if(optimizer_name == 'Adam'):
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
  if(optimizer_name == 'RMSprop'):
     optimizer = optim.RMSprop(model.parameters(), lr=learning_rate)
  if(optimizer_name == 'SGD'):
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)

  # TRAINING LOOP
  for epoch in range(epochs):
    model.train()
    for data_features, data_labels in train_loader:
      # Cast data_features to float32 to match model's expected dtype
      data_features = data_features.to(device).float()
      data_labels = data_labels.to(device).float().unsqueeze(1)

      output = model(data_features)
      loss = criterion(output, data_labels)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

  # EVALUATION after training loop
  model.eval()
  total_correct = 0
  total_samples = 0
  with torch.no_grad():
      for data_features_test, data_labels_test in test_loader:
          # Cast data_features_test to float32 to match model's expected dtype
          data_features_test = data_features_test.to(device).float()
          data_labels_test = data_labels_test.to(device).float().unsqueeze(1)

          test_outputs = model(data_features_test)
          predicted_labels = (torch.sigmoid(test_outputs) > 0.5).squeeze()

          total_correct += (predicted_labels == data_labels_test.squeeze()).sum().item()
          total_samples += data_labels_test.size(0)

  accuracy = total_correct / total_samples
  return accuracy

In [90]:
!pip install optuna

In [93]:
import optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

[I 2025-12-16 13:25:15,783] A new study created in memory with name: no-name-89d8c161-8c42-429a-9916-e6448dd1e2d2
[I 2025-12-16 13:28:29,827] Trial 0 finished with value: 0.9993153330290369 and parameters: {'hidden_layer': 3, 'neurons': 56, 'epchs': 5, 'lr': 7.942858763648484e-05, 'dropout': 0.2, 'batch_size': 16, 'optimizer': 'Adam'}. Best is trial 0 with value: 0.9993153330290369.
[I 2025-12-16 13:30:46,668] Trial 1 finished with value: 0.9982795547909132 and parameters: {'hidden_layer': 5, 'neurons': 128, 'epchs': 3, 'lr': 2.9122976565388394e-05, 'dropout': 0.4, 'batch_size': 16, 'optimizer': 'SGD'}. Best is trial 0 with value: 0.9993153330290369.
[I 2025-12-16 13:32:42,158] Trial 2 finished with value: 0.9993328885923949 and parameters: {'hidden_layer': 2, 'neurons': 104, 'epchs': 7, 'lr': 3.650072147888721e-05, 'dropout': 0.30000000000000004, 'batch_size': 32, 'optimizer': 'Adam'}. Best is trial 2 with value: 0.9993328885923949.
[I 2025-12-16 13:34:17,717] Trial 3 finished with va